In [14]:
import matplotlib.pyplot as plt
import sys
import numpy as np
import seaborn as sns
import duckdb
import pandas as pd
import plotly.express as px
import statistics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [15]:
diamonds = duckdb.connect('./data/diamonds_train.db')
diamonds

In [ ]:
duckdb.sql('INSTALL sqlite; LOAD sqlite;')

In [ ]:
tabla = diamonds.sql('describe').df()
tabla

In [ ]:
tabla.shape

In [ ]:
tabla.info()

## Registro tablas

In [ ]:
tabla_city = diamonds.sql('SELECT * from diamonds_city').df()
tabla_city

In [ ]:
tabla_clarity = diamonds.sql('SELECT * from diamonds_clarity').df()
tabla_clarity

In [ ]:
tabla_color = diamonds.sql('SELECT * from diamonds_color').df()
tabla_color

In [ ]:
tabla_cut = diamonds.sql('SELECT * from diamonds_cut').df()
tabla_cut

In [ ]:
tabla_dimensions = diamonds.sql('SELECT * from diamonds_dimensions').df()
tabla_dimensions

In [ ]:
tabla_properties = diamonds.sql('SELECT * from diamonds_properties').df()
tabla_properties

In [ ]:
tabla_transactional = diamonds.sql('SELECT * from diamonds_transactional').df()
tabla_transactional

## Merge de los dataframes

In [ ]:
tabla_price = pd.merge(tabla_dimensions, tabla_transactional, how='inner', on = 'index_id')
tabla_price[:5]

In [ ]:
tabla_city = pd.merge(tabla_price, tabla_city, how='inner', on = 'city_id')
tabla_city[:5]

In [ ]:
tabla_ids = pd.merge(tabla_city, tabla_properties, how='inner', on = 'index_id')
tabla_ids[:5]

In [ ]:
tabla_clar = pd.merge(tabla_ids, tabla_clarity, how='inner', on = 'clarity_id')
tabla_clar[:5]

In [ ]:
tabla_color = pd.merge(tabla_clar, tabla_color, how='inner', on = 'color_id')
tabla_color[:5]

In [ ]:
tabla_final = pd.merge(tabla_color, tabla_cut, how='inner', on = 'cut_id')
tabla_final[:5]

In [ ]:
train = tabla_final[['depth', 'table', 'x', 'y', 'z', 'price', 'carat', 'city', 'clarity', 'color', 'cut']]
train

In [ ]:
train.describe()

In [ ]:
numerical = train.select_dtypes(include=['float64', 'int64'])
numerical

In [ ]:
categorical = train.select_dtypes(include=['object'])
categorical

## Data Visualization

In [ ]:
train.plot(kind='box', figsize=(15,10), subplots=True, layout=(3,3))
plt.show()

In [ ]:
sns.histplot(x=numerical["price"]);


In [ ]:
sns.barplot(data=train, x="cut", y="price")

In [ ]:
x = train.plot.hexbin(x='price',
                         y='carat',
                         gridsize=30,
                         sharex=False,     
                         figsize=(10, 8))
x.set_xlabel('price')
x.set_ylabel('carat')
plt.tight_layout()

In [ ]:
corr = train.select_dtypes(include='number').corr()
sns.heatmap(corr, cmap='coolwarm', annot=True)
plt.show()

## Transform data

### Encoding

In [ ]:
test = pd.read_csv('./data/diamonds_test.csv')
test

In [ ]:
target = 'price'
cat_features = ['cut', 'color', 'clarity']
num_features = ['carat', 'depth', 'table', 'x', 'y', 'z']

for cat_feat in cat_features:
    train[cat_feat] = train[cat_feat].astype('category')
    test[cat_feat] = test[cat_feat].astype('category')
    
cat_df = pd.get_dummies(train[cat_features])
num_df = train.loc[:,num_features]
train_df = pd.concat([cat_df, num_df], axis=1)

cat_df = pd.get_dummies(test[cat_features])
num_df = test.loc[:,num_features]
test_df = pd.concat([cat_df, num_df], axis=1)


features = list(cat_df.columns) + list(num_df.columns)

In [ ]:
train_df

In [ ]:
boolean_columns = ['cut_Fair', 'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good',
       'color_D', 'color_E', 'color_F', 'color_G', 'color_H', 'color_I',
       'color_J', 'clarity_I1', 'clarity_IF', 'clarity_SI1', 'clarity_SI2',
       'clarity_VS1', 'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2']

for col in boolean_columns:
    train_df[col] = train_df[col].astype(int)

In [ ]:
train_df

In [ ]:
boolean_columns = ['cut_Fair', 'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good',
       'color_D', 'color_E', 'color_F', 'color_G', 'color_H', 'color_I',
       'color_J', 'clarity_I1', 'clarity_IF', 'clarity_SI1', 'clarity_SI2',
       'clarity_VS1', 'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2']

for col in boolean_columns:
    test_df[col] = test_df[col].astype(int)

In [ ]:
test_df

### Scaling

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(train_df.loc[:,features].values)
y = train[target]

In [ ]:
print(X.shape,y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")
print(f"X_train: {type(X_train)}, X_test: {type(X_test)}, y_train: {type(y_train)}, y_test: {type(y_test)}")

### Modeling

### Cat Boost Regressor

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
catboost = CatBoostRegressor()

In [ ]:
model_catboost = catboost.fit(X_train, y_train)
y_pred_catboost = catboost.predict(X_test)
mean_squared_error(y_test, y_pred_catboost)**0.5

In [ ]:
y_pred_catboost

In [ ]:
X_predict = scaler.transform(test_df.loc[:,features].values)

# testing our model vs X_predict
y_hat = catboost.predict(X_predict)

In [ ]:
y_hat

In [ ]:
errors = cross_val_score(catboost, X, y, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
np.mean(-errors)

In [ ]:
submission_6 = pd.DataFrame({'id': test['id'], 'price': y_hat})
submission_6.to_csv('submission_6.csv', index=False)

### GradientBoostingRegressor

In [ ]:
# Resultado: 548.59164

params = {'n_estimators': 50,
          'max_depth': 10,
          'learning_rate': 0.16,
          'loss': 'squared_error'}

In [ ]:
reg = GradientBoostingRegressor(**params)
model_gb = reg.fit(X_train, y_train)
y_pred_gb = reg.predict(X_test)
mean_squared_error(y_test, y_pred_gb)**0.5

In [ ]:
model_gb = reg.fit(X, y)

In [ ]:
X_predict = scaler.transform(test_df.loc[:,features].values)

# testing our model vs X_predict
y_hat = reg.predict(X_predict)

In [ ]:
errors = cross_val_score(reg, X, y, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
np.mean(-errors)

### Linear Regression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred_lr = model.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred_lr)**0.5

In [ ]:
X_predict = scaler.transform(test_df.loc[:,features].values)

# testing our model vs X_predict
y_hat = model.predict(X_predict)

In [ ]:
errors = cross_val_score(model, X, y, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
np.mean(-errors)

### MLP Regressor

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
ann = MLPRegressor()

In [ ]:
model_ann = ann.fit(X_train, y_train)
y_pred_ann = ann.predict(X_test)
mean_squared_error(y_test, y_pred_ann)**0.5

In [ ]:
y_pred_ann

In [ ]:
X_predict = scaler.transform(test_df.loc[:,features].values)

# testing our model vs X_predict
y_hat = ann.predict(X_predict)

In [ ]:
y_hat

In [ ]:
errors = cross_val_score(ann, X, y, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
np.mean(-errors)

### RandomForestRegressor

In [ ]:
rf = RandomForestRegressor()

In [ ]:
model_rf = rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
mean_squared_error(y_test, y_pred_rf)**0.5

In [ ]:
X_predict = scaler.transform(test_df.loc[:,features].values)

# testing our model vs X_predict
y_hat = rf.predict(X_predict)

In [ ]:
errors = cross_val_score(rf, X, y, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
np.mean(-errors)

### SVR

In [ ]:
from sklearn.svm import SVR

In [ ]:
svm = SVR()

In [ ]:
model_svm = svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
rmse = mean_squared_error(y_test, y_pred_svm)**0.5

In [ ]:
X_predict = scaler.transform(test_df.loc[:, features].values)
y_hat_svm = svm.predict(X_predict)

In [ ]:
errors = cross_val_score(svm, X, y, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
np.mean(-errors)

### Xgb Regressor

In [ ]:
!pip install xgboost
from xgboost import XGBRegressor

xgbr = XGBRegressor(verbosity=0) 
model_xgbr = xgbr.fit(X_train, y_train)
y_pred_xgbr = xgbr.predict(X_test)
mean_squared_error(y_test, y_pred_xgbr)**0.5

In [ ]:
X_predict = scaler.transform(test_df.loc[:,features].values)

# testing our model vs X_predict
y_hat = xgbr.predict(X_predict)

In [ ]:
from sklearn.metrics import mean_squared_error
errors = cross_val_score(xgbr, X, y, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
rmse_cv = (-errors.mean()) ** 0.5

In [ ]:
np.mean(-errors)